In [1]:
import anndata
import umap
import xgboost
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import pickle
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import display, Image, HTML
import mlflow
import re
regex = re.compile(r"perturbation_experiment_([^_]+)_level")

# base definition and original adata:
if not "adata" in locals():
    levels = ["0", "0.5", "1.0"]
    data_dir = "/data/gene_perturbation_colon_epithelial_scGPT_with_binning_1"
    file_pattern = data_dir + "/perturbation_experiment_{gene}_level_{level}"
    base_fname = file_pattern.format(gene = "JUND", level = "1.0")
    base_embeddings = np.load(f"{base_fname}.embeddings.npy")
    adata = anndata.read_h5ad(f"{base_fname}.h5ad")
    base_df = adata.obs[["tissue", "cell_type", 'sex', 'development_stage',"disease"]].copy()
data_dir = "/data/colon_epithelial_cellxgene_perturbation_4"
file_pattern = data_dir + "/perturbations/perturbation_experiment_{gene}_level_{level}"
genes = set([regex.search(f.name).group(1) for f in list(Path(data_dir+"/perturbations").glob("*.npz"))])
model_filename = "cgboost_on_emebeddings_with_binning_predict_disease_2024-05-05.pkl"
with open(model_filename, "rb") as f:
    model = pickle.load(f)

# load npz file:
# np.load(f"{data_dir}/perturbations/perturbation_experiment_CEACAM5_level_0.embeddings.npz")["emb"].shape

base_df.sample(5, random_state=0)

,tissue,cell_type,sex,development_stage,disease
cell_id,,,,,
N128624_E-GTAGATCAGCTGTTCA,colonic epithelium,enterocyte,female,fifth decade human stage,Crohn disease
N21_Epi_A-CTGAATCTAAGGCG,left colon,enterocyte,female,human adult stage,normal
N124246_L-AACAACCAGTATTAGG,lamina propria of mucosa of colon,brush cell,male,fourth decade human stage,Crohn disease
N175041_N2-TTAGGACGTGGGTATG,left colon,enterocyte,male,seventh decade human stage,Crohn disease
N51_Epi_B-GCATGCGGTCTGCGGT,caecum,enterocyte,male,human adult stage,normal


In [ ]:
# check the correlations between the categorical variables in base_df with cramer's correlation:
